# Imports

In [1]:
# Python imports
import urllib.request, json 
import pandas as pd
from pandas.io.json import json_normalize
from datetime import date, datetime

In [2]:
now = datetime.now()

# Coleta de dados via api

In [3]:
# Referências
## https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos
## https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao

with urllib.request.urlopen("https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoAnuais?$filter=baseCalculo%20eq%200%20and%20Data%20ge%20'2022-07-01'%20and%20DataReferencia%20ge%20'2023'&$format=json&$select=Indicador,IndicadorDetalhe,Data,DataReferencia,Mediana") as url:
    annual_data = json.loads(url.read().decode())

# Transformação .json para df

In [4]:
annual_data_df = pd.DataFrame.from_dict(annual_data['value'], orient='columns')
annual_df = annual_data_df.query("Indicador in ('IPCA', 'Taxa de desocupação', 'Câmbio', 'IGP-M', 'Selic', 'PIB Total')").sort_values(by=["DataReferencia","Data"])
annual_df['rank_order'] = annual_df.groupby(['Indicador', 'DataReferencia'])['Data'].rank(method='first', ascending=False)
annual_df = annual_df.query("rank_order == 1 or rank_order == 15 or rank_order == 30 or rank_order == 45 or rank_order == 60")[['Indicador', 'Data', 'DataReferencia', 'Mediana']].sort_values(by=['Indicador','DataReferencia', 'Data'])

# Pivotar colunas

In [5]:
annual_df_pivoted = annual_df.pivot(index=["Data", "DataReferencia"], columns=["Indicador"], values = ["Mediana"])
annual_df_pivoted.columns = [' '.join(col) for col in annual_df_pivoted.columns.values]

annual_df_pivoted.reset_index(inplace=True)
annual_df_pivoted.rename(columns = {'Data':'projection_version', 'DataReferencia':'reference_year', 
                                     'Mediana Câmbio':'exchange_rate', 'Mediana IGP-M':'igpm_rate', 
                                     'Mediana IPCA':'ipca_rate', 'Mediana PIB Total':'gdp_rate', 
                                     'Mediana Selic':'selic_rate', 'Mediana Taxa de desocupação':'unemployment_rate'}, inplace = True)

annual_df_pivoted['last_update'] = now
annual_df_pivoted.insert(0, 'last_update', annual_df_pivoted.pop('last_update'))

annual_df_pivoted['projection_version'].replace('-','_',inplace=True, regex=True)
annual_df_pivoted['projection_version'] = "v_" + annual_df_pivoted['projection_version']

## Display

In [6]:
annual_df_pivoted

,last_update,projection_version,reference_year,exchange_rate,igpm_rate,ipca_rate,gdp_rate,selic_rate,unemployment_rate
0,2023-05-31 13:49:06.839832,v_2023_03_01,2023,5.2500,4.1486,5.9013,0.8300,12.750,8.700
1,2023-05-31 13:49:06.839832,v_2023_03_01,2024,5.3000,4.2000,4.0215,1.5000,10.000,8.900
2,2023-05-31 13:49:06.839832,v_2023_03_01,2025,5.3000,4.0000,3.8000,1.8000,9.000,8.930
3,2023-05-31 13:49:06.839832,v_2023_03_01,2026,5.3500,4.0000,3.7650,2.0000,8.500,9.000
4,2023-05-31 13:49:06.839832,v_2023_03_01,2027,5.3500,4.0000,4.0000,NaN,8.500,NaN
5,2023-05-31 13:49:06.839832,v_2023_03_22,2023,5.2500,4.0734,5.9545,0.8775,12.750,8.700
6,2023-05-31 13:49:06.839832,v_2023_03_22,2024,5.3000,4.2000,4.0763,1.4250,10.000,9.000
7,2023-05-31 13:49:06.839832,v_2023_03_22,2025,5.3000,4.0000,3.9000,1.7000,9.000,9.000
8,2023-05-31 13:49:06.839832,v_2023_03_22,2026,5.4000,4.0000,4.0000,1.7500,9.000,9.000
9,2023-05-31 13:49:06.839832,v_2023_03_22,2027,5.4000,4.0000,4.0000,1.8500,8.875,9.000


## Exportação para .xlsx

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
annual_df_pivoted.to_excel(excel_writer=r'/content/gdrive/MyDrive/expectativas_macro.xlsx', index=False)